provision_retail_space_sqm: Предоставление торговых площадей

provision_retail_space_modern_sqm: Обеспечение населения торговыми площадями современных форматов, кв.м

retail_trade_turnover_per_cap: Оборот розничной торговли на душу населения

turnover_catering_per_cap: Оборот сферы общественного питания на человека

theaters_viewers_per_1000_cap: Количество зрителей театров на 1000 человек населения количество зрительских мест в российских театрах Министерства культуры на 100 000 человек населения : Общее количество зрительских мест в российских театрах Министерства культуры на 100 000 человек населения

museum_visits_per_100_cap: Количество посещений музеев на 1000 человек населения пропускная способность спортивных сооружений

population_reg_sports_share: Доля населения, регулярно занимающегося спортом

students_reg_sports_share: Доля школьников и студентов, регулярно занимающихся спортом, в общем количестве

apartment_build: Строительство городских жилых домов

apartment_fund_sqm: Городской жилой фонд

In [20]:
import pandas as pd
import random as rn
import numpy as np
import os

# Визуализация
%matplotlib inline
import matplotlib.pyplot as plt

# Метрики
from sklearn.metrics import mean_squared_log_error


In [21]:
# загрзка обучающего датасета
data_train = pd.read_csv("train.csv")

# загрузка проверочного датасета
data_test = pd.read_csv("test.csv")

In [25]:
print(data_train.shape)

(30471, 292)


### Выбор столбцов (признаков) для обучения

price_doc: цена продажи (это целевая переменная)

id: идентификатор транзакции
 
timestamp: дата транзакции

full_sq: общая площадь в квадратных метрах, включая лоджии, балконы и другие нежилые помещения

life_sq: жилая площадь в квадратных метрах, за исключением лоджий, балконов и других нежилых помещений

floor: для квартир, этаж здания

max_floor: количество этажей в

material: материал стен

build_year: год постройки

num_room: количество жилых комнат

kitch_sq: площадь кухни

state: состояниеквартиры 

product_type: покупка для частного использования или инвестиция

sub_area: название района

In [14]:
# просмотр столбцов для определения признаков для обучения
list(data_test.columns.values.tolist())

['id',
 'timestamp',
 'full_sq',
 'life_sq',
 'floor',
 'max_floor',
 'material',
 'build_year',
 'num_room',
 'kitch_sq',
 'state',
 'product_type',
 'sub_area',
 'area_m',
 'raion_popul',
 'green_zone_part',
 'indust_part',
 'children_preschool',
 'preschool_quota',
 'preschool_education_centers_raion',
 'children_school',
 'school_quota',
 'school_education_centers_raion',
 'school_education_centers_top_20_raion',
 'hospital_beds_raion',
 'healthcare_centers_raion',
 'university_top_20_raion',
 'sport_objects_raion',
 'additional_education_raion',
 'culture_objects_top_25',
 'culture_objects_top_25_raion',
 'shopping_centers_raion',
 'office_raion',
 'thermal_power_plant_raion',
 'incineration_raion',
 'oil_chemistry_raion',
 'radiation_raion',
 'railroad_terminal_raion',
 'big_market_raion',
 'nuclear_reactor_raion',
 'detention_facility_raion',
 'full_all',
 'male_f',
 'female_f',
 'young_all',
 'young_male',
 'young_female',
 'work_all',
 'work_male',
 'work_female',
 'ekder_all'

In [ ]:
data_X = data_train['']

data_y = 

In [12]:
# проверка пропусков на обучающем датасете
data_X.info()    #.columns.values  # = df[df['meter_reading'] < 250000]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30471 entries, 0 to 30470
Columns: 292 entries, id to price_doc
dtypes: float64(119), int64(157), object(16)
memory usage: 67.9+ MB
